In [1]:
import sys
sys.path.append('../')
import daq
import file
import globalVAR as Gvar
import visa
import numpy as np
import time
from random import randint
import PyCapture2 as pc2

%load_ext autoreload
%autoreload 1

In [2]:
DAQ = daq.Daq(desc='Triple probe test in air plasma.')
DAQ.connect_instr('TDS2024C', 1)
DAQ.connect_instr('KA3005P', 'ttyUSB0')

Power supply ID: KORADKA3005PV2.0
Device ttyUSB0 successfully connected and process started.
Oscilloscope ID: TEKTRONIX,TDS 2024C,C046401,CF:91.1CT FV:v24.26

Device C046401 successfully connected and process started.


In [3]:
DAQ.send_command(DAQ.command_queue['ttyUSB0'], 'turn_on')
DAQ.send_command(DAQ.command_queue['C046401'], 'acquire_off')
DAQ.send_command(DAQ.command_queue['C046401'], 'set_acquisition_stop', ("SEQuence",))
DAQ.send_command(DAQ.command_queue['C046401'], 'set_acquisition_mode', ("AVErage",))
DAQ.send_command(DAQ.command_queue['C046401'], 'set_average_num', (128,))

In [4]:
v = np.linspace(2.0, 30, 57)

for i in range(len(v)):
    for j in [1,2]:
        DAQ.send_command(DAQ.command_queue['C046401'], 'set_data_source', ('CH{}'.format(j),))
        time.sleep(3)
        DAQ.send_command(DAQ.command_queue['ttyUSB0'], 'set_record_voltage', (v[i],))
        time.sleep(0.1)
        DAQ.send_command(DAQ.command_queue['C046401'], 'acquire_on')
        time.sleep(15)
        DAQ.send_command(DAQ.command_queue['C046401'], 'save_waveform')
        time.sleep(3)
        #DAQ.send_command(DAQ.command_queue['C046401'], 'set_data_source', ('CH<2>',))
        #time.sleep(1)
        #DAQ.send_command(DAQ.command_queue['C046401'], 'save_waveform')
        #time.sleep(3)

In [59]:
DAQ.send_command(DAQ.command_queue['ttyUSB0'], 'set_voltage', (30,))

In [5]:
DAQ.send_command(DAQ.command_queue['ttyUSB0'], 'turn_off')

In [6]:
DAQ.end_dataset()

In [9]:
for key in DAQ.instr:
    for elem in DAQ.instr[key]:
        DAQ.disconnect_instr(key, elem)